In [1]:
import requests
import csv
import numpy as np
import pandas as pd
from datetime import date
import glob
import os
import json
import time

In [2]:
crime_response = requests.get("https://phl.carto.com/api/v2/sql?filename=incidents_part1_part2&format=csv&q=SELECT%20*%20,%20ST_Y(the_geom)%20AS%20lat,%20ST_X(the_geom)%20AS%20lng%20FROM%20incidents_part1_part2%20WHERE%20dispatch_date_time%20%3E=%20%272022-01-01%27%20AND%20dispatch_date_time%20%3C%20%272023-01-01%27")
crime_text = crime_response.text.strip().split("\n")
crime_reader = csv.reader(crime_text)
crime_list=list(crime_reader)

#REMOVING USELESS INFORMATION AND CLEANING ROWS
newarr=[]
for list_item in crime_list[1:]:
    if list_item[14] == '' or list_item[15] == '' or list_item[16] =='' or list_item[17] == '':
        continue
    else:
        newarr.append(list_item)
headings = crime_list[:1][0]
newarr.insert(0,headings)
df = pd.DataFrame(newarr[1:], columns=headings)

df.drop(['the_geom','cartodb_id','the_geom_webmercator','objectid','dc_dist','psa','dispatch_date_time','dc_key','ucr_general','point_x','point_y'], axis = 1, inplace = True)
today = date.today()
d1 = today.strftime("%d/%m/%Y")
fileName = str(d1) + ".csv"
df.to_csv("../data/11-24-22.csv")

In [3]:
csv_files = glob.glob('*.{}'.format('csv'))
csv_files

[]

In [4]:
df_append = pd.DataFrame()
#append all files together
for file in csv_files[:-1]:
            df_temp = pd.read_csv(file)
            df_append = df_append.append(df_temp, ignore_index=True)
df_append

""


In [ ]:
df_append = df_append.drop_duplicates()
df_append = df_append.drop(['Unnamed: 0','hour_'],axis=1)

In [ ]:
df_append = df_append.sort_values(by=['dispatch_date'],ignore_index=True)
df_append

In [ ]:
df_append.to_csv("Cleaned-Crime.csv")

In [ ]:
sv_files = glob.glob('*.{}'.format('csv'))
crimeDataFile = csv_files[-2]

crimeData = pd.read_csv(crimeDataFile)

requiredDates=list(set(crimeData['dispatch_date'].tolist()))
requiredDates.sort()
requiredDates

In [ ]:
weather_details = {}



def get_weather(date):
    
    weather_response = requests.get('http://api.weatherapi.com/v1/history.json?key=2eec0b76d1b24fe28c5164334222511&q=Philadelphia&dt='+date)
    weather_data = weather_response.json()
    dailyForecast = weather_data['forecast']['forecastday'][0]['day']
    astro = weather_data['forecast']['forecastday'][0]['astro']
    hourlyForecast = weather_data['forecast']['forecastday'][0]['hour']
    weather_details[date] = [dailyForecast,astro,hourlyForecast]
        
        
    

for date in requiredDates:
    get_weather(date)
    
weather_details

In [ ]:
updated_weather_details={}
for i in weather_details:
    item = weather_details[i]
    dailyForecast = item[0]
    astro = item[1]
    hourlyForecast = item[2]
    
    #Daily Forecast
    maxtemp_f = dailyForecast['maxtemp_f']
    mintemp_f = dailyForecast['mintemp_f']
    avgtemp_f = dailyForecast['avgtemp_f']
    maxwind_mph = dailyForecast['maxwind_mph']
    avghumidity = dailyForecast['avghumidity']
    totalprecip_mm = dailyForecast['totalprecip_mm']
    condition = dailyForecast['condition']['text']
    
    forecastDict = {'maxTemp' : maxtemp_f,
                'minTemp' : mintemp_f,
                'avgTemp' : avgtemp_f,
                'maxWind' : maxwind_mph,
                'avgHumidity' : avghumidity,
                'totalPrecipitation':totalprecip_mm,
                'condition': condition
               }
    
    
    #Astro
    sunrise = astro['sunrise']
    sunset = astro['sunset']
    moonrise = astro['moonrise']
    moonset = astro['moonset']
    moonphase = astro['moon_phase']
    
    astroDict = {
            'sunrise' : sunrise,
            'sunset' : sunset,
            'moonrise' : moonrise,
            'moonset' : moonset,
            'moonphase' : moonphase
    } 
    
    #Hourly Forecast
    
    updated_weather_details[i] = [forecastDict, astroDict,hourlyForecast]

    
updated_weather_details  

In [ ]:
weather = pd.DataFrame()
# stockDF = pd.DataFrame(columns)


df = pd.DataFrame(crimeData)
time = df['dispatch_time'].tolist()
headings = ['Temperature','Maximum Temperature', 'Average Temperature','Maximum Wind','Average Humidity','Total Precipitation', 'Condition','Sunrise', 'Sunset','Moonrise','Moonset','Moon Phase']

weather = pd.DataFrame(columns=headings)

for j, ele in enumerate(crimeData['dispatch_date']):
    
    weather_info = updated_weather_details[ele]
    
    daily = weather_info[0]
    astro = weather_info[1]
    hourly = weather_info[2]
    time = int(crimeData.iloc[j]['dispatch_time'][:2])
    
    todayData = hourly[time]
    
    temp = todayData['temp_f']
    
    
    
    
    values = {'Temperature' : temp
              ,'Maximum Temperature' : daily['maxTemp'],
              'Average Temperature' : daily['avgTemp'],
              'Maximum Wind' : daily['maxWind'],
              'Average Humidity' : daily['avgHumidity'],
              'Total Precipitation' : daily['totalPrecipitation'],
              'Condition' : todayData['condition']['text'],
              'Sunrise' : astro['sunrise'],
              'Sunset' : astro['sunset'],
              'Moonrise' : astro['moonrise'],
              'Moonset' : astro['moonset'],
              'Moon Phase' : astro['moonphase']
             }

    information = pd.DataFrame([values],columns=headings)
    weather = weather.append(information,ignore_index = True)
    

weather
weather.to_csv("Weather.csv")

In [ ]:
weatherData = pd.read_csv("Weather.csv")
crimeWeather = pd.concat([crimeData, weatherData], axis=1)


crimeWeather = crimeWeather.drop(['Unnamed: 0'],axis=1)
crimeWeather[:10]

In [ ]:
crimeWeather.to_csv("Crime-Weather.csv")

In [6]:
holidayUrl = 'https://holidayapi.com/v1/holidays?pretty&key=e1df5793-02a5-4982-bf6d-df30497e8305&country=US&year=2021'

holidayResponse = requests.get(holidayUrl).json()

holidays = holidayResponse['holidays']

holidays

def filteredHolidays(holiday):
        
    date = holiday['date']
    dateArray = date.split("-")
    year = int(dateArray[0])+1
    newDate = str(year)+'-'+dateArray[1]+'-'+dateArray[2]
    public = "NOT A PUBLIC HOLIDAY"
    
    if(holiday['public']==True):
        public = "PUBLIC HOLIDAY"
        
        
    updatedHolidays = {
        'Holiday' : holiday['name'],
        'Date' : newDate,
        'Public Holiday' : public,
    }
    
    return updatedHolidays

updatedHolidays = map(filteredHolidays,holidays)

updatedHolidays = list(updatedHolidays)

In [7]:
holidayDataFrame = pd.DataFrame(updatedHolidays)

holidayDataFrame = holidayDataFrame[:-1]
# holidayDataFrame
updatedHolidays

[{'Holiday': "New Year's Day",
  'Date': '2022-01-01',
  'Public Holiday': 'PUBLIC HOLIDAY'},
 {'Holiday': 'Seventh Day of Kwanzaa',
  'Date': '2022-01-01',
  'Public Holiday': 'NOT A PUBLIC HOLIDAY'},
 {'Holiday': 'Epiphany',
  'Date': '2022-01-06',
  'Public Holiday': 'NOT A PUBLIC HOLIDAY'},
 {'Holiday': 'Orthodox Christmas Eve',
  'Date': '2022-01-06',
  'Public Holiday': 'NOT A PUBLIC HOLIDAY'},
 {'Holiday': 'Orthodox Christmas Day',
  'Date': '2022-01-07',
  'Public Holiday': 'NOT A PUBLIC HOLIDAY'},
 {'Holiday': 'Stephen Foster Memorial Day',
  'Date': '2022-01-13',
  'Public Holiday': 'NOT A PUBLIC HOLIDAY'},
 {'Holiday': "Orthodox New Year's Day",
  'Date': '2022-01-14',
  'Public Holiday': 'NOT A PUBLIC HOLIDAY'},
 {'Holiday': 'Martin Luther King Jr. Day',
  'Date': '2022-01-18',
  'Public Holiday': 'PUBLIC HOLIDAY'},
 {'Holiday': 'National Freedom Day',
  'Date': '2022-02-01',
  'Public Holiday': 'NOT A PUBLIC HOLIDAY'},
 {'Holiday': 'Groundhog Day',
  'Date': '2022-02-02',


In [8]:
def holiday_dates(holidayDict):
    return holidayDict['Date']

holidayDates = list(map(holiday_dates,updatedHolidays))
holidayDates

['2022-01-01',
 '2022-01-01',
 '2022-01-06',
 '2022-01-06',
 '2022-01-07',
 '2022-01-13',
 '2022-01-14',
 '2022-01-18',
 '2022-02-01',
 '2022-02-02',
 '2022-02-05',
 '2022-02-07',
 '2022-02-12',
 '2022-02-14',
 '2022-02-15',
 '2022-02-16',
 '2022-02-17',
 '2022-02-22',
 '2022-03-01',
 '2022-03-02',
 '2022-03-05',
 '2022-03-14',
 '2022-03-17',
 '2022-03-20',
 '2022-03-28',
 '2022-03-29',
 '2022-04-01',
 '2022-04-01',
 '2022-04-02',
 '2022-04-03',
 '2022-04-04',
 '2022-04-05',
 '2022-04-06',
 '2022-04-13',
 '2022-04-15',
 '2022-04-19',
 '2022-04-21',
 '2022-04-22',
 '2022-04-22',
 '2022-04-30',
 '2022-04-30',
 '2022-04-30',
 '2022-05-01',
 '2022-05-01',
 '2022-05-01',
 '2022-05-01',
 '2022-05-01',
 '2022-05-02',
 '2022-05-03',
 '2022-05-04',
 '2022-05-05',
 '2022-05-06',
 '2022-05-06',
 '2022-05-07',
 '2022-05-08',
 '2022-05-09',
 '2022-05-13',
 '2022-05-15',
 '2022-05-15',
 '2022-05-15',
 '2022-05-19',
 '2022-05-21',
 '2022-05-22',
 '2022-05-23',
 '2022-05-24',
 '2022-05-25',
 '2022-05-

In [9]:
def get_holiday_Data(d):
    
    for data in updatedHolidays:
        if data['Date'] == str(d):
            return data
        else:
            continue
            
    value = {'Holiday': "NOT A HOLIDAY",
              'Date': str(d),
             'Public Holiday': 'NOT A HOLIDAY'
                    }
    return value
get_holiday_Data('2022-01-13')

{'Holiday': 'Stephen Foster Memorial Day',
 'Date': '2022-01-13',
 'Public Holiday': 'NOT A PUBLIC HOLIDAY'}

In [ ]:
holidayHeading = ['Holiday','Date','Public Holiday']
